In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

from fastai.text import *
from fastai.imports import *
from fastai.text import *
from fastai import *

In [ ]:
#loading the datasets

#data_path = "/Users/amalnair/Documents/Amal-WorkSpace/MachineHack_Competitions/MovieScriptsClassification/Data"

train = pd.read_csv('/kaggle/input/MovieScriptsParticipantsData/Train.csv')
test = pd.read_csv('/kaggle/input/MovieScriptsParticipantsData/Test.csv')


scripts_path = '/kaggle/input/MovieScriptsParticipantsData/Scripts/'

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
os.listdir(scripts_path)

In [ ]:
len(os.listdir(scripts_path))

In [ ]:
#Looking inside a script file
print("File Name : ", train['File_Name'].values[0])
print("-"*60)
print("Length of content : ", len(open(scripts_path + os.sep + train['File_Name'].values[0], "r").read()))
print("-"*60)
print(open(scripts_path + os.sep + train['File_Name'].values[0], "r").read())

In [ ]:
# Loading the scripts to dataframe
train['Script'] = [open(scripts_path + os.sep + file, "r").read() for file in train['File_Name']]
test['Script'] = [open(scripts_path + os.sep + file, "r").read() for file in test['File_Name']]

In [ ]:
test

In [ ]:
train

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
#training set

print("Unique number of classes in train : ", train['Labels'].nunique())

In [ ]:
train['Labels'].value_counts().plot(kind = 'bar',figsize=(7,10), rot=0, yticks=range(0,420,20))

In [ ]:
train["Script_Length"] = train['Script'].apply(lambda x: len(x))
test["Script_Length"] = test['Script'].apply(lambda x: len(x))

In [ ]:
train

In [ ]:
train.groupby('Labels').mean().plot(kind='line', xticks = range(0,22,1), rot = 0, title="Mean Script Length Per Class")
train.groupby('Labels').median().plot(kind='line', xticks = range(0,22,1), rot = 0, title="Median Script Length Per Class")
train.groupby('Labels').min().plot(kind='line', xticks = range(0,22,1), rot = 0, title="Minimum Script Length Per Class")
train.groupby('Labels').max().plot(kind='line', xticks = range(0,22,1), rot = 0, title="Maximum Script Length Per Class")

In [ ]:
train['Labels'].value_counts()

In [ ]:
# Creating training and validation data
from sklearn.model_selection import train_test_split

temp = train.drop(train[train['Labels'] == 18].index[0], axis = 0)

xtrain, xvalid, ytrain, yvalid = train_test_split(temp[['Script']].values, temp.Labels.values, 
                                                  stratify=temp.Labels.values, 
                                                  random_state=42, 
                                                  test_size=0.3, shuffle=True)


In [ ]:
len(set(yvalid))

In [ ]:
print(xtrain.shape)
print(ytrain.shape)

In [ ]:
print(xvalid.shape)
print(yvalid.shape)

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english', 
                      max_features=100, 
                      analyzer='word', 
                      token_pattern=r'\w{1,}', #re for breaking a sentence into words
                      strip_accents='unicode',
                      ngram_range=(1, 3),
                      use_idf=1, 
                      smooth_idf=1, 
                      sublinear_tf=1,
                      min_df=3)  #minimum document frequency

In [ ]:
# Fitting TF-IDF to both training and test sets (semi-supervised learning)
temp = list(xtrain) + list(xvalid)
temp = np.asarray(temp).reshape(-1)
tfidf.fit(temp)

In [ ]:
xtrain_tfv =  tfidf.transform(xtrain.reshape(-1)) 
xvalid_tfv = tfidf.transform(xvalid.reshape(-1))

In [ ]:
xtrain_tfv.shape

In [ ]:
xvalid_tfv.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)

clf.fit(xtrain_tfv, ytrain)


In [ ]:
predictions = clf.predict_proba(xvalid_tfv)

In [ ]:
print("logloss: %0.3f " % log_loss(yvalid, predictions))

In [ ]:
xtrain = train.Script.values
ytrain = train.Labels.values

xtest = test.Script.values

In [ ]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

In [ ]:
tfidf_c = TfidfVectorizer(stop_words = 'english', 
                      max_features=100, 
                      analyzer='word', 
                      token_pattern=r'\w{1,}', #re for breaking a sentence into words
                      strip_accents='unicode',
                      ngram_range=(1, 3),
                      use_idf=1, 
                      smooth_idf=1, 
                      sublinear_tf=1,
                      min_df=3)  #minimum document frequency

In [ ]:
tfidf_c.fit(xtrain.reshape(-1))

In [ ]:
xtrain_tfv =  tfidf_c.transform(xtrain)
xtest_tfv = tfidf_c.transform(xtest)

In [ ]:
xtrain_tfv.shape

In [ ]:
xtest_tfv.shape

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf_2 = clf = LogisticRegression(C=1.0)
clf_2.fit(xtrain_tfv, ytrain)

In [ ]:
predictions_test = clf_2.predict_proba(xtest_tfv)

In [ ]:
pd.DataFrame(predictions_test)

In [ ]:
pred = pd.DataFrame(predictions_test)

In [ ]:
pred['File_Name'] = test['File_Name']

In [ ]:
pred.head()

In [ ]:
col_order = [i for i in range(0,22)]
col_order.insert(0, 'File_Name')
pred = pred[col_order]

In [ ]:
pred.head()

In [ ]:
pred.columns

In [ ]:
submission = pd.DataFrame({
       
        "File_Name": pred['File_Name'],
        "0": pred[0],
        "1": pred[1],
        "2": pred[2],
        "3": pred[3],
        "4": pred[4],
        "5": pred[5],
        "6": pred[6],
        "7": pred[7],
        "8": pred[8],
        "9": pred[9],
        "10": pred[10],
        "11": pred[11],
        "12": pred[12],
        "13": pred[13],
        "14": pred[14],
        "15": pred[15],
        "16": pred[16],
        "17": pred[17],
        "18": pred[18],
        "19": pred[19],
        "20": pred[20],
        "21": pred[21]
                

    })
submission.to_csv('./mf.csv', index=False)
print(submission)

In [ ]:
type(xtrain_tfv)

In [ ]:
from scipy import sparse 
xtrain_tfv_scount = sparse.hstack((xtrain_tfv,train.Script_Length.values[:,None])).A

In [ ]:
xtrain_tfv_scount.shape

In [ ]:
xtest_tfv_scount = sparse.hstack((xtest_tfv,test.Script_Length.values[:,None])).A

In [ ]:
xtest_tfv_scount.shape

In [ ]:
#scaling
"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(xtrain_tfv_scount)
sc.fit_transform(xtest_tfv_scount)
"""

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf_3 = LogisticRegression(C=1.0)
clf_3.fit(xtrain_tfv_scount, ytrain)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf_3 = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
clf_3.fit(xtrain_tfv_scount, ytrain)

In [ ]:
predictions_test_1 = clf_3.predict_proba(xtest_tfv_scount)

In [ ]:
pred2 = pd.DataFrame(predictions_test_1)

In [ ]:
pred2.head()

In [ ]:
pred2['File_Name'] = test['File_Name']

In [ ]:
pred2 = pred2[col_order]

In [ ]:
pred2.head(2)

In [ ]:
pred=pred2

In [ ]:
submission = pd.DataFrame({
       
        #"File_Name": pred['File_Name'],
        "0": pred[0],
        "1": pred[1],
        "2": pred[2],
        "3": pred[3],
        "4": pred[4],
        "5": pred[5],
        "6": pred[6],
        "7": pred[7],
        "8": pred[8],
        "9": pred[9],
        "10": pred[10],
        "11": pred[11],
        "12": pred[12],
        "13": pred[13],
        "14": pred[14],
        "15": pred[15],
        "16": pred[16],
        "17": pred[17],
        "18": pred[18],
        "19": pred[19],
        "20": pred[20],
        "21": pred[21]
                

    })
submission.to_csv('./mc.csv', index=False)
print(submission)